In [10]:
%pip install imagecodecs
%pip install numpy
%pip install matplotlib
%pip install csbdeep
%pip install pandas
%pip install seaborn
%pip install tensorflow
%pip install keras

NameError: name 'list_slides' is not defined

In [8]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = "retina"

from tifffile import imread, imsave
from csbdeep.utils import Path, normalize
from csbdeep.utils.tf import keras_import
keras = keras_import()

from stardist import export_imagej_rois, random_label_cmap
from stardist.models import StarDist2D

np.random.seed(0)
cmap = random_label_cmap()


In [ ]:
def show_image(img, **kwargs):
    fig, ax = plt.subplots(2,4, figsize=(16,8))
    mid = [s//2+600 for s in img.shape[:2]]
    for a,t,u in zip(ax.ravel(),[1,2,4,8,16,32,64,128],[16,8,4,2,1,1,1,1]):
        sl = tuple(slice(c-s//t//2, c+s//t//2, u) for s,c in zip(img.shape[:2], mid))
        a.imshow(img[sl], **kwargs)
        a.axis('off')
    plt.tight_layout()
    plt.show()